/bin/bash: python3106: command not found


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt", use_fast=True)

2023-03-09 10:29:24.568576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 10:29:32.400375: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib
2023-03-09 10:29:32.401651: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

ImportError: You need to install sacremoses to use BioGptTokenizer. See https://pypi.org/project/sacremoses/ for installation.

In [ ]:
import pandas as pd
df = pd.read_csv('../data/df_xy.csv', index_col=False)

In [ ]:
df = df[["text"]]
df = df.head(50000)

In [ ]:
from datasets import Dataset

In [5]:
ds = Dataset.from_pandas(df)

In [6]:
ds

Dataset({
    features: ['text'],
    num_rows: 50000
})

In [7]:
def tokenize(batch):
    return tokenizer(batch["text"], return_tensors="pt", padding=True, truncation=True)

In [8]:
ds = ds.map(tokenize, num_proc=4, batched=True)

In [9]:
ds = ds.remove_columns(["text"])

In [10]:
tts_ds = ds.train_test_split(test_size=0.3)

In [11]:
tts_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 15000
    })
})

In [12]:
block_size = 128

In [13]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
lm_datasets = tts_ds.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [15]:
tokenizer.decode(lm_datasets["train"][17]["input_ids"])

'<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad></s>Abdominal pain Other- previous ulcer Failed intubation There is a stricture in the third part of the duodenum which is HhpertrophiedScope not held by stricture. The duodenal stricture has a smooth and regular contour, without any signs of ulceration or erosion.The ulcer was biopsied x8 for further analysis.. character (0) FOLLOW UP: The patient should avoid consuming too much fat, as this can delay gastric emptying and delay the healing of the ulcer. RECOMMENDATION: The patient should be instructed'

In [16]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("microsoft/biogpt").to('cuda')

In [17]:
from transformers import Trainer, TrainingArguments

In [18]:
training_args = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    output_dir="models",
    report_to="none"
)

In [19]:
import wandb

In [20]:
#wandb.init(mode="disabled")

In [21]:
wandb.init(project='open_medicine')

AttributeError: module 'wandb' has no attribute 'init'

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],

)

In [ ]:
trainer.train()

/opt/conda/envs/python3106/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 87887
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 32958
  Number of trainable parameters = 346763264


Epoch,Training Loss,Validation Loss


Saving model checkpoint to models/checkpoint-500
Configuration saved in models/checkpoint-500/config.json
Configuration saved in models/checkpoint-500/generation_config.json
Model weights saved in models/checkpoint-500/pytorch_model.bin
Saving model checkpoint to models/checkpoint-1000
Configuration saved in models/checkpoint-1000/config.json
Configuration saved in models/checkpoint-1000/generation_config.json
Model weights saved in models/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to models/checkpoint-1500
Configuration saved in models/checkpoint-1500/config.json
Configuration saved in models/checkpoint-1500/generation_config.json
Model weights saved in models/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to models/checkpoint-2000
Configuration saved in models/checkpoint-2000/config.json
Configuration saved in models/checkpoint-2000/generation_config.json
Model weights saved in models/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to models/checkp

In [3]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [2]:
import torch
torch.cuda.is_available()

True